In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import PIL

# IMPORT PATCH NET



# NOTE: This is a hack to get around "User-agent" limitations when downloading MNIST datasets
#       see, https://github.com/pytorch/vision/issues/3497 for more information
from six.moves import urllib
opener = urllib.request.build_opener()
opener.addheaders = [('User-agent', 'Mozilla/5.0')]
urllib.request.install_opener(opener)


In [ ]:
epsilons = [] #[0, .05, .1, .15, .2, .25, .3, .35, .4] #, -.5, -.6, -.7, -.8, -.9, -1]
#pretrained_model = "/content/lenet_mnist_model.pth"
use_cuda=True

In [ ]:
#from PIL import Image
from matplotlib import cm

def CutoutAbs(img, v):  # [0, 60] => percentage: [0, 0.2]
    # assert 0 <= v <= 20
    if v < 0:
        return img
    #w, h = img.size
    w = 28
    h = 28
    x0 = np.random.uniform(w)
    y0 = np.random.uniform(h)

    #print(type(img))
    #print(np.shape(img))
    #plt.imshow(img, cmap="gray") # before patch is added


    x0 = int(max(0, x0 - v / 2.))
    y0 = int(max(0, y0 - v / 2.))
    x1 = min(w, x0 + v)
    y1 = min(h, y0 + v)
    
    for i in range(len(img)):
      if (i >= x0 and i <= x1):
        for j in range(len(img[i])):
          if (j >= y0 and j <= y1):
            img[i][j] = 0.5 #128 #(128, 128, 128)    

    #plt.imshow(img, cmap="gray") # after patch is added
    
    #xy = (x0, y0, x1, y1)
    #color = (128, 128, 128) # gray
    # color = (0, 0, 0)
    #img = img.copy()
    #im = Image.fromarray(np.uint8(cm.gist_earth(img)*255))
    #PIL.ImageDraw.Draw(im).rectangle(xy, color)
    #img = np.array(im)
    #plt.imshow(img, cmap="gray")
    #print(np.shape(img))
    return img, x0, y0, x1, y1

In [ ]:
# FGSM attack code
def fgsm_attack(image, epsilon, data_grad):
    print('in fgsm. image shape:',np.shape(image))
    print('')
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    sign_data_grad = torch.squeeze(sign_data_grad)
    print('in fgsm. epsilon*sign_data_grad shape:', np.shape(epsilon*sign_data_grad))
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [ ]:
#from PIL import Image
'''from matplotlib import cm
from PIL import Image, ImageOps
def CutoutAbsWithPerturbation(img, v, pos_epsilon_for_attack, data_grad):  # [0, 60] => percentage: [0, 0.2]
    # assert 0 <= v <= 20
    print('data_grad:', np.shape(data_grad))
    
    if v < 0:
        return img
    #w, h = img.size
    w = 28
    h = 28
    x0 = np.random.uniform(w)
    y0 = np.random.uniform(h)

    #print(type(img))
    print(np.shape(img))
    #plt.imshow(img, cmap="gray") # before patch is added
 
    gray_img = Image.new('RGB', (5, 5), color = 'gray')
    grayscale_img = ImageOps.grayscale(gray_img)
    patch_img_no_perturbation = np.array(grayscale_img)
    print('patch_img_no_perturbation shape:', np.shape(patch_img_no_perturbation))
    #patch_img_no_perturbation = [[0]*5 for i in range(5)] #[[]] # 5 by 5 gray image
    #for i in range(v):
      #for j in range(v):
        #patch_img_no_perturbation[i][j] = 0.5 # gray color is b/t 0 and 1.
    

    x0 = int(max(0, x0 - v / 2.)) # x0, y0 becomes bottom left
    y0 = int(max(0, y0 - v / 2.))
    x1 = min(w, x0 + v) # x1, y1 becomes bottom right
    y1 = min(h, y0 + v)

    patch_img_no_perturbation_torch = torch.from_numpy(patch_img_no_perturbation)
    
    perturbed_patch_img = fgsm_attack(patch_img_no_perturbation_torch, pos_epsilon_for_attack, data_grad)
    
    for i in range(len(img)):
      if (i >= x0 and i <= x1):
        for j in range(len(img[i])):
          if (j >= y0 and j <= y1):
            img[i][j] = .5 #perturbed_patch_img[i-x0][j-y0] #128 #(128, 128, 128) 

    #plt.imshow(img, cmap="gray") # after patch is added
    
    #xy = (x0, y0, x1, y1)
    #color = (128, 128, 128) # gray
    # color = (0, 0, 0)
    #img = img.copy()
    #im = Image.fromarray(np.uint8(cm.gist_earth(img)*255))
    #PIL.ImageDraw.Draw(im).rectangle(xy, color)
    #img = np.array(im)
    #plt.imshow(img, cmap="gray")
    #print(np.shape(img))
    return img'''

'from matplotlib import cm\nfrom PIL import Image, ImageOps\ndef CutoutAbsWithPerturbation(img, v, pos_epsilon_for_attack, data_grad):  # [0, 60] => percentage: [0, 0.2]\n    # assert 0 <= v <= 20\n    print(\'data_grad:\', np.shape(data_grad))\n    \n    if v < 0:\n        return img\n    #w, h = img.size\n    w = 28\n    h = 28\n    x0 = np.random.uniform(w)\n    y0 = np.random.uniform(h)\n\n    #print(type(img))\n    print(np.shape(img))\n    #plt.imshow(img, cmap="gray") # before patch is added\n \n    gray_img = Image.new(\'RGB\', (5, 5), color = \'gray\')\n    grayscale_img = ImageOps.grayscale(gray_img)\n    patch_img_no_perturbation = np.array(grayscale_img)\n    print(\'patch_img_no_perturbation shape:\', np.shape(patch_img_no_perturbation))\n    #patch_img_no_perturbation = [[0]*5 for i in range(5)] #[[]] # 5 by 5 gray image\n    #for i in range(v):\n      #for j in range(v):\n        #patch_img_no_perturbation[i][j] = 0.5 # gray color is b/t 0 and 1.\n    \n\n    x0 = int(ma

In [ ]:
pretrained_model = "/content/patch_net.pth" #"/content/lenet_mnist_model.pth"
use_cuda=True

In [ ]:
print("CUDA Available: ",torch.cuda.is_available())
device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")

CUDA Available:  False


In [ ]:
# LeNet Model definition
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10) #(_, 2) means 2 classes (has patch, or no patch).

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [ ]:
# Define what device we are using
print("CUDA Available: ",torch.cuda.is_available())
#device = torch.device("cuda" if (use_cuda and torch.cuda.is_available()) else "cpu")


# FOR TESTING/VALIDATION: Set the model in evaluation mode. In this case this is for the Dropout layers
#model.eval()

CUDA Available:  False


In [ ]:
test_loader = torch.utils.data.DataLoader(  # 10,000
    datasets.MNIST('../data', train=False, download=True, transform=transforms.Compose([
            transforms.ToTensor(),
            ])),
        batch_size=1, shuffle=True)
#print("len of test", len(test_loader)) 
#train_loader = torch.utils.data.DataLoader( # 60,000
    #datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
     #       transforms.ToTensor(),
      #      ])),
       # batch_size=1, shuffle=True)
#print("len of test 2", len(train_loader))

#allmnist = [] # original 10,000 images w/o patch
#allmnistpatch = [] # all 10,000 images w/ patches

counter = 0

# TESTING

criterion = nn.CrossEntropyLoss()
adversarial_test_images = [] # 20,000 images. 10,000 original images, 10,000 adversarially attacked perturbed images. 

# Initialize the network
model = Net()
model.fc2 = nn.Linear(50, 2)

model.load_state_dict(torch.load(pretrained_model, map_location='cpu'))

model.to(device)


# Load the pretrained model
pretrained_model = "/content/patch_net.pth"

for param in model.parameters():
    param.requires_grad = True

correct = 0
for data, labels in test_loader: # 10,000 images. data is a tensor.
  labels = torch.tensor([1])
  counter+=1
  img = data.squeeze().numpy() # only 1 image b/c batch_size = 1.
  print(np.shape(img))
  print('length of image', len(img))
  #adversarial_test_images.append(img) # original image. part of all test images (with patch and w/o patch)
  #print(img.size)
  imgWithPatch, x0, y0, x1, y1 = CutoutAbs(img, 5) # gray patch coordinates

  pos_eps_for_attack = .5
  patch_data = torch.from_numpy(imgWithPatch).float() # convert back to tensor. data is the image w/ patch

  patch_data, labels = patch_data.to(device), labels.to(device) # puts data on gpu

  # Set requires_grad attribute of tensor. Important for Attack
  patch_data.requires_grad = True
  
  #data, target = data.to(device), target.to(device)
  #data.requires_grad = True
  #print('data:', data)
  print('data shape:', np.shape(patch_data))
  patch_data = torch.unsqueeze(patch_data, 0)
  print('data shape:', np.shape(patch_data))
  patch_data = torch.unsqueeze(patch_data, 0) # now data is 4d which model requires
  print('data shape:', np.shape(patch_data))
  #print('data:', data)

  output = model(patch_data)
  
  print('output shsape', np.shape(output))
  init_pred = output.max(1, keepdim=True)[1]
  print('label shape', np.shape(labels))
  #labels = labels.unsqueeze(0)
  #print('label shape', np.shape(labels))
  print('output shape', np.shape(output))
  print('output type', type(output))
  print('output:', output)
  print('labels:', labels)
  #print('F.log_softmax(output):', F.log_softmax(output))
  #print('type of F.log_softmax(output):', type(F.log_softmax(output)))
  #print('shape of F.log_softmax(output):', np.shape(F.log_softmax(output)))
  loss = F.nll_loss(output, labels)

  # Zero all existing gradients
  model.zero_grad()
  
  # Calculate gradients of model in backward pass
  loss.backward()
  print('output.grad', output.grad)
  #print('data_grad', np.shape(data_grad), type(data_grad), data_grad)
  print('data:', patch_data, 'data.grad:', patch_data.grad)
  data_grad = patch_data.grad.data
  print('data_grad shape before zeroing', np.shape(data_grad))

  # keep patch gradients, zero everything else
  for i in range(len(data_grad)):
    for j in range(len(data_grad[0])):
      if i < x0 or i > x1 or j > y0 or j < y1:
        data_grad[i][j] = 0

  pos_epsilon_for_attack = .5
  perturbed_patch_data = fgsm_attack(patch_data, pos_epsilon_for_attack, data_grad)

  # Forward pass the data through the model
  outputs = model(perturbed_patch_data)

  #apply sigmoid or softmax
  sf = nn.Softmax(dim=1) #makes items in a row add to 1; dim = 0 makes items in a column add to 1
  outputs = sf(outputs)
  # Get predictions from the maximum value
  _, predicted = torch.max(outputs, 1)
  
  #add labels + predictions to full set for auroc later
  if torch.cuda.is_available():
      outs_ones = outputs.detach().cpu().numpy()[:, 1]
      labelsnp = labels.cpu().numpy()
  else:
      outs_ones = outputs.detach().numpy()[:, 1]
      labelsnp = labels.numpy()

  test_all_labels = np.append(test_all_labels, labelsnp)
  test_all_probs_ones = np.append(test_all_probs_ones, outs_ones)

  #  USE GPU FOR MODEL
  # Total correct predictions
  try:
    if torch.cuda.is_available():
        correct += (predicted.cpu() == labels.cpu()).sum()#(thresh_predicted == tlabels.cpu()).sum()
    else:
        correct += (predicted == labels).sum()
  except:
    print("??")
    correct += (predicted == labelsnp).sum()

  # Total number of labels
  try:
      counter += len(predicted.cpu())#labels.size(0)
  except:
      print("len 0 labels", labels)
  accuracy = 100 * correct # counter
  print("accuracy for iteration {} = {}. {} correct out of count {}".format(i, accuracy, correct, counter))



  

(28, 28)
length of image 28
data shape: torch.Size([28, 28])
data shape: torch.Size([1, 28, 28])
data shape: torch.Size([1, 1, 28, 28])
output shsape torch.Size([1, 2])
label shape torch.Size([1])
output shape torch.Size([1, 2])
output type <class 'torch.Tensor'>
output: tensor([[-7.7624e+00, -4.2549e-04]], grad_fn=<LogSoftmaxBackward>)
labels: tensor([1])
output.grad None
data: tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
     

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:89: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:91: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more information.
/usr/local/lib/python3.7/dist-packages/ipykernel

AttributeError: ignored

In [ ]:
loss_arr = []

In [ ]:
import torch.optim as optim
import torch.utils.data as data
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.utils import resample
import random

class DatasetWithPatch(data.Dataset):    
    def __init__(self, allimages, allimagespatch, alllabels, alllabelspatch, phase, transform=None):
        """Data loader for Model."""
        super(DatasetWithPatch, self).__init__()
        
        #train, val, test
        self.phase = phase
        myimgs = []
        all_labels = []
        startind = 0
        endind = 0
        print("hello", len(allimages))
        if (self.phase == "train"):
          endind = 5000 #int(.6*len(allimages)) # 6000
        elif (self.phase == "val"):
          startind = int(.6*len(allimages))
          endind = int(.8*len(allimages))
        else: #test
          startind = 0 #int(.8*len(allimages))
          endind = 1000 #int(len(allimages))adversarial_test_images
          for i in range(startind,endind):
            #myimgs.append(allimages[i])
            myimgs.append(adversarial_test_images[i])
            #all_labels.append(alllabels[i])
            all_labels.append(1) # labels are all 1s
        print('phase:', self.phase)
        print('myimgs length:', len(myimgs))
        print('all_labels length:', len(all_labels))
        print("startind", startind)
        print("endind", endind)
        
        for i in range(startind,endind):
          myimgs.append(allimages[i])
          myimgs.append(allimagespatch[i])
          all_labels.append(alllabels[i])
          all_labels.append(alllabelspatch[i])
        print("length of myimgs", len(myimgs))
        print("length of all_labels", len(all_labels))
        #self.imgs = myimgs
        #self.all_labels = all_labels

        #shuffle data here
        #group images and albels together
        #shuffle
        #ungroup
        temp = list(zip(myimgs, all_labels))
        random.shuffle(temp)
        myimgs, all_labels = zip(*temp)
        self.imgs = myimgs
        self.all_labels = all_labels


    
    #getitem is called 'batch_size' number of times in one iteration of the epoch
    def __getitem__(self, i):
        img_frame = self.imgs[i] #3 stacked frames (rgb) from same patient OR same image x3        
        #create label for image
        label = torch.LongTensor(1)
        #print('label type[i]: ', type(self.all_labels[i]))
        #print('label[i]: ', self.all_labels[i])
        #print('length of label[i]: ', len(self.all_labels[i]))
        label[0] = int(self.all_labels[i])

        input1 = torch.from_numpy(img_frame).float()            
        return {'input': input1, 'label': label}

    def __len__(self):
        return len(self.imgs)

In [ ]:
# calls init in DatasetWithPatch
#train_set = DatasetWithPatch(allimages=allmnist, allimagespatch=allmnistpatch, 
 #                alllabels=labels, alllabelspatch=labelspatch, phase="train", transform=None)
#train_set_loader = DataLoader(dataset=train_set, batch_size=16, shuffle=False)


In [ ]:
loss_arr = []

In [ ]:
#train(model, device, train_set_loader)

In [ ]:
#patch_path = '/content/patch_net.pth'
#test(patch_path) # path to patch_net

In [ ]:
#np.shape(test_loader)
#shape(test_loader)